In [2]:
def predictgen(predict_date):
    import os
    import pandas as pd
    import json
    import datetime

    def createDayDataframe(filename):
        path = "tongsdata/"
        filedate = filename.split(".")[0].split("-")

        with open(path + filename, 'r', encoding='utf-8') as fin:
            data_json = json.loads(fin.read())

        #  Create X DataFrame
        todayData = {time: data_json[time]['currently'] for time in data_json}
        todayData_DF = pd.DataFrame(todayData).T

        #  Create y DataFrame
        energyBalanceFilename = "energyBalance/Energy_Balance_{}_{}_{}.csv".format(filedate[0], filedate[1], filedate[2]) #change due to json filename
        energyBalance = pd.read_csv(path + energyBalanceFilename, delimiter=";")

        energyBalance_DF = pd.DataFrame(index=energyBalance.iloc[:-1,0].apply(lambda x: x[2:-1]))
        energyBalance_DF['PV Power Generation'] = energyBalance['PV power generation / Mean values [W]  '].values[:-1]
        energyBalance_DF = energyBalance_DF[energyBalance_DF != ' ']
        energyBalance_DF = energyBalance_DF.dropna()

        #  Join X y DataFrame
        data = todayData_DF.join(energyBalance_DF)
        return data

    def cleanDataframe(masterDF):
        masterData_cleaned = masterDF.reset_index()
        masterData_cleaned = masterData_cleaned.rename(columns={'index': 'time_of_day'})
        masterData_cleaned = masterData_cleaned[masterData_cleaned['PV Power Generation'].notna()]
        masterData_cleaned['month'] = masterData_cleaned['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
        masterData_cleaned['cloudCover'] = masterData_cleaned['cloudCover'].fillna(value=masterData_cleaned['cloudCover'].mean())
        masterData_cleaned['PV Power Generation'] = masterData_cleaned['PV Power Generation'].apply(lambda x: x.replace(",", "")).astype('float')
        return masterData_cleaned

    def featureEngineering(masterDF):
        masterDF = masterDF.drop(columns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure'])
        masterDF = pd.get_dummies(masterDF, columns=['time_of_day', 'precipType', 'uvIndex', 'month']) 
        return masterDF

    

    def pvpredict(predict_date) :
        path = "tongsdata/"
        filename = predict_date.replace("_",'-') + '.json'

        with open(path + filename, 'r', encoding='utf-8') as fin:
            data_json = json.loads(fin.read())

        todayData = {time: data_json[time]['currently'] for time in data_json}
        todayData_DF = pd.DataFrame(todayData).T
        todayData_DF.index.names=['time_of_day']
        todayData_DF = todayData_DF.loc['06:30':'18:00']
        todayData_DF = todayData_DF.reset_index()
        todayData_DF['month'] = todayData_DF['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
        #todayData_DF['cloudCover'] = todayData_DF['cloudCover'].fillna(value=todayData_DF['cloudCover'].mean())
        dropcolumns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure','cloudCover']
        for i in dropcolumns:
            if i in todayData_DF.columns:
                todayData_DF = todayData_DF.drop(columns=[i])
        todayData_DF = pd.get_dummies(todayData_DF, columns=['time_of_day', 'precipType', 'uvIndex', 'month'])

        masterData_for_predict = masterData.drop(columns=['PV Power Generation'])
        for i in range(len(masterData_for_predict.columns)) : #drop power balance
            if masterData_for_predict.columns[i] not in todayData_DF.columns:
                todayData_DF.insert(i, masterData.columns[i+1], 0)
        missingvalue = todayData_DF[todayData_DF.isna().any(axis=1)].index
        import math
        for i in missingvalue:
            for j in todayData_DF.columns:
                if math.isnan(todayData_DF.loc[i,j]) :
                    todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j]+todayData_DF.loc[i+1,j])/2
        first_predict = todayData_DF.astype('float').values

        results_first_predict = exported_pipeline.predict(first_predict)
        results_first_predict = np.array(results_first_predict).tolist()
        for i in range(25):
            results_first_predict.insert(0,0.0)
        for j in range (24):
            results_first_predict.append(0.0)
        return results_first_predict

    Generation=pvpredict(predict_date)
    return Generation

## แก้

In [1]:
def createDayDataframe(filename):
    path = "tongsdata/"
    filedate = filename.split(".")[0].split("-")

    with open(path + filename, 'r', encoding='utf-8') as fin:
        data_json = json.loads(fin.read())

    #  Create X DataFrame
    todayData = {time: data_json[time]['currently'] for time in data_json}
    todayData_DF = pd.DataFrame(todayData).T

    #  Create y DataFrame
    energyBalanceFilename = "energyBalance/Energy_Balance_{}_{}_{}.csv".format(filedate[0], filedate[1], filedate[2]) #change due to json filename
    energyBalance = pd.read_csv(path + energyBalanceFilename, delimiter=";")

    energyBalance_DF = pd.DataFrame(index=energyBalance.iloc[:-1,0].apply(lambda x: x[2:-1]))
    energyBalance_DF['PV Power Generation'] = energyBalance['PV power generation / Mean values [W]  '].values[:-1]
    energyBalance_DF = energyBalance_DF[energyBalance_DF != ' ']
    energyBalance_DF = energyBalance_DF.dropna()

    #  Join X y DataFrame
    data = todayData_DF.join(energyBalance_DF)
    return data

def cleanDataframe(masterDF):
    masterData_cleaned = masterDF.reset_index()
    masterData_cleaned = masterData_cleaned.rename(columns={'index': 'time_of_day'})
    masterData_cleaned = masterData_cleaned[masterData_cleaned['PV Power Generation'].notna()]
    masterData_cleaned['month'] = masterData_cleaned['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
    masterData_cleaned['cloudCover'] = masterData_cleaned['cloudCover'].fillna(value=masterData_cleaned['cloudCover'].mean())
    masterData_cleaned['PV Power Generation'] = masterData_cleaned['PV Power Generation'].apply(lambda x: x.replace(",", "")).astype('float')
    return masterData_cleaned

def featureEngineering(masterDF):
    masterDF = masterDF.drop(columns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure','cloudCover'])
    masterDF = pd.get_dummies(masterDF, columns=['time_of_day', 'precipType', 'uvIndex', 'month']) 
    return masterDF

    

In [2]:
def pvpredict(predict_date) :
    path = "tongsdata/"
    filename =     predict_date.replace("_",'-') + '.json'
    
    with open(path + filename, 'r', encoding='utf-8') as fin:
        data_json = json.loads(fin.read())
    
    todayData = {time: data_json[time]['currently'] for time in data_json}
    todayData_DF = pd.DataFrame(todayData).T
    todayData_DF.index.names=['time_of_day']
    todayData_DF = todayData_DF.loc['06:30':'18:00']
    todayData_DF = todayData_DF.reset_index()
    todayData_DF['month'] = todayData_DF['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
    #todayData_DF['cloudCover'] = todayData_DF['cloudCover'].fillna(value=todayData_DF['cloudCover'].mean())
    dropcolumns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure','cloudCover']

    for i in dropcolumns:
        if i in todayData_DF.columns:
            todayData_DF = todayData_DF.drop(columns=[i])
    todayData_DF = pd.get_dummies(todayData_DF, columns=['time_of_day', 'precipType', 'uvIndex', 'month'])
    
    masterData_for_predict = masterData.drop(columns=['PV Power Generation'])
    for i in range(len(masterData_for_predict.columns)) : #drop power balance
        if masterData_for_predict.columns[i] not in todayData_DF.columns:
            todayData_DF.insert(i, masterData.columns[i+1], 0)
    missingvalue = todayData_DF[todayData_DF.isna().any(axis=1)].index
    import math
    for i in missingvalue:
        for j in todayData_DF.columns:
            if   i == 0:    #add for index error case                                         
                    todayData_DF.loc[i,j] = (todayData_DF.loc[i+1,j])
            elif i+1 > len(todayData_DF.index)-1:               
                    todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j])
            elif math.isnan(todayData_DF.loc[i,j]) :
                todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j]+todayData_DF.loc[i+1,j])/2
    first_predict = todayData_DF.astype('float').values
    
    results_first_predict = exported_pipeline.predict(first_predict)
    results_first_predict = np.array(results_first_predict).tolist()
    for i in range(25):
        results_first_predict.insert(0,0.0)
    for j in range (24):
        results_first_predict.append(0.0)
    return results_first_predict

### รันแค่ครั้งเดียวเพื่อ fit model

In [3]:
import os
import pandas as pd
import json
import datetime

files = os.listdir("tongsdata/")

#files.remove(".DS_Store")
files.remove("energyBalance")

masterData = pd.DataFrame()

for file in files:
    r = createDayDataframe(file).loc['06:30':'18:00']
    #print(file, r.shape)
    masterData = pd.concat([masterData, r], axis=0)

masterData = cleanDataframe(masterData)
masterData = featureEngineering(masterData)

features = list(masterData.drop(columns=['PV Power Generation']).columns)

masterData = masterData.dropna()

X = masterData.drop(columns=['PV Power Generation']).astype('float').values
y = masterData['PV Power Generation'].astype('float').values


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-91867.4890210794
exported_pipeline = RandomForestRegressor(bootstrap=False, max_features=0.3, min_samples_leaf=1, min_samples_split=7, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

#from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
#print(mean_squared_error(testing_target, results), 
#mean_absolute_error(testing_target, results), 
#r2_score(testing_target, results))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [17]:
datelist=[]
for i in range(1,31):
    datelist.append('2018_11_'+str(i).zfill(2))
datelist    

['2018_11_01',
 '2018_11_02',
 '2018_11_03',
 '2018_11_04',
 '2018_11_05',
 '2018_11_06',
 '2018_11_07',
 '2018_11_08',
 '2018_11_09',
 '2018_11_10',
 '2018_11_11',
 '2018_11_12',
 '2018_11_13',
 '2018_11_14',
 '2018_11_15',
 '2018_11_16',
 '2018_11_17',
 '2018_11_18',
 '2018_11_19',
 '2018_11_20',
 '2018_11_21',
 '2018_11_22',
 '2018_11_23',
 '2018_11_24',
 '2018_11_25',
 '2018_11_26',
 '2018_11_27',
 '2018_11_28',
 '2018_11_29',
 '2018_11_30']

In [18]:
genlist=[]
for i in datelist:
    genlist.append(pvpredict(i))

## TEST

In [5]:
predictgen('2018_11_02')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



KeyError: 'cloudCover'

In [11]:
import os
import pandas as pd
import json
import datetime

In [31]:
predict_date = '2018_12_01'
path = "tongsdata/"
filename = predict_date.replace("_",'-') + '.json'

with open(path + filename, 'r', encoding='utf-8') as fin:
    data_json = json.loads(fin.read())

todayData = {time: data_json[time]['currently'] for time in data_json}
todayData_DF = pd.DataFrame(todayData).T

In [32]:
todayData_DF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96 entries, 00:00 to 23:45
Data columns (total 14 columns):
apparentTemperature    96 non-null object
cloudCover             96 non-null object
dewPoint               96 non-null object
humidity               96 non-null object
icon                   96 non-null object
precipType             96 non-null object
summary                96 non-null object
temperature            96 non-null object
time                   96 non-null object
uvIndex                96 non-null object
visibility             96 non-null object
windBearing            96 non-null object
windGust               15 non-null object
windSpeed              96 non-null object
dtypes: object(14)
memory usage: 11.2+ KB


In [33]:
todayData_DF.index.names=['time_of_day']
todayData_DF = todayData_DF.loc['06:30':'18:00']
todayData_DF = todayData_DF.reset_index()
todayData_DF['month'] = todayData_DF['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
todayData_DF

,time_of_day,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipType,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,month
0,06:30,82.96,0.19,74.45,0.86,clear-day,rain,Humid,79.14,1543620600,0,3.87,18,NaN,2.06,12
1,06:45,83.33,0.19,74.44,0.85,clear-day,rain,Humid,79.32,1543621500,0,3.87,14,NaN,1.98,12
2,07:00,83.69,0.19,74.44,0.85,clear-day,rain,Humid,79.5,1543622400,0,3.87,11,NaN,1.91,12
3,07:15,84.52,0.22,74.28,0.83,clear-day,rain,Humid,79.95,1543623300,0,3.87,17,NaN,2.68,12
4,07:30,85.29,0.26,74.1,0.81,partly-cloudy-day,rain,Humid and Partly Cloudy,80.4,1543624200,0,3.87,21,NaN,3.48,12
5,07:45,86.01,0.3,73.92,0.8,partly-cloudy-day,rain,Humid and Partly Cloudy,80.85,1543625100,1,3.87,23,NaN,4.28,12
6,08:00,86.68,0.34,73.71,0.78,partly-cloudy-day,rain,Humid and Partly Cloudy,81.3,1543626000,1,3.87,25,NaN,5.09,12
7,08:15,87.99,0.34,73.91,0.77,partly-cloudy-day,rain,Humid and Partly Cloudy,82.02,1543626900,1,4.09,24,NaN,4.39,12
8,08:30,89.28,0.34,74.09,0.75,partly-cloudy-day,rain,Humid and Partly Cloudy,82.74,1543627800,2,4.31,22,NaN,3.69,12
9,08:45,90.54,0.34,74.27,0.74,partly-cloudy-day,rain,Humid and Partly Cloudy,83.46,1543628700,2,4.53,20,NaN,2.99,12


In [34]:
#todayData_DF['cloudCover'] = todayData_DF['cloudCover'].fillna(value=todayData_DF['cloudCover'].mean())
dropcolumns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure','cloudCover']
for i in dropcolumns:
    if i in todayData_DF.columns:
        todayData_DF = todayData_DF.drop(columns=[i])
todayData_DF = pd.get_dummies(todayData_DF, columns=['time_of_day', 'precipType', 'uvIndex', 'month'])

masterData_for_predict = masterData.drop(columns=['PV Power Generation'])
for i in range(len(masterData_for_predict.columns)) : #drop power balance
    if masterData_for_predict.columns[i] not in todayData_DF.columns:
        todayData_DF.insert(i, masterData.columns[i+1], 0)
missingvalue = todayData_DF[todayData_DF.isna().any(axis=1)].index
import math
for i in missingvalue:
    for j in todayData_DF.columns:
        if   i == 0:    #add for index error case                                         
                todayData_DF.loc[i,j] = (todayData_DF.loc[i+1,j])
        elif i+1 > len(todayData_DF.index)-1:               
                todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j])
        elif math.isnan(todayData_DF.loc[i,j]) :
            todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j]+todayData_DF.loc[i+1,j])/2
first_predict = todayData_DF.astype('float').values

In [35]:
todayData_DF

,apparentTemperature,cloudCover,dewPoint,humidity,ozone,precipIntensity,precipProbability,pressure,temperature,visibility,...,uvIndex_4,uvIndex_5,uvIndex_6,uvIndex_7,uvIndex_8,uvIndex_9,month_1,month_10,month_11,month_12
0,82.96,0,74.45,0.86,0,0,0,0,79.14,3.87,...,0,0,0,0,0,0,0,0,0,1
1,83.33,0,74.44,0.85,0,0,0,0,79.32,3.87,...,0,0,0,0,0,0,0,0,0,1
2,83.69,0,74.44,0.85,0,0,0,0,79.5,3.87,...,0,0,0,0,0,0,0,0,0,1
3,84.52,0,74.28,0.83,0,0,0,0,79.95,3.87,...,0,0,0,0,0,0,0,0,0,1
4,85.29,0,74.1,0.81,0,0,0,0,80.4,3.87,...,0,0,0,0,0,0,0,0,0,1
5,86.01,0,73.92,0.8,0,0,0,0,80.85,3.87,...,0,0,0,0,0,0,0,0,0,1
6,86.68,0,73.71,0.78,0,0,0,0,81.3,3.87,...,0,0,0,0,0,0,0,0,0,1
7,87.99,0,73.91,0.77,0,0,0,0,82.02,4.09,...,0,0,0,0,0,0,0,0,0,1
8,89.28,0,74.09,0.75,0,0,0,0,82.74,4.31,...,0,0,0,0,0,0,0,0,0,1
9,90.54,0,74.27,0.74,0,0,0,0,83.46,4.53,...,0,0,0,0,0,0,0,0,0,1


In [21]:
results_first_predict = exported_pipeline.predict(first_predict)
results_first_predict = np.array(results_first_predict).tolist()
for i in range(25):
    results_first_predict.insert(0,0.0)
for j in range (24):
    results_first_predict.append(0.0)
results_first_predict

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 310.49766666666676,
 318.4728333333334,
 329.85716666666684,
 367.56816666666674,
 340.7268333333334,
 736.9731666666668,
 748.9095000000002,
 784.0850000000003,
 873.6976666666665,
 861.8316666666666,
 942.7780000000001,
 975.3708333333336,
 962.4143333333335,
 1049.068666666667,
 1074.5908333333336,
 1072.869666666667,
 1258.385166666667,
 1369.1015000000002,
 1651.976166666667,
 1651.976166666667,
 1651.976166666667,
 1645.0113333333336,
 1645.0113333333336,
 1629.2448333333336,
 1605.2015000000001,
 1640.6163333333334,
 1638.5078333333338,
 1642.3536666666669,
 1637.7210000000007,
 1641.3076666666668,
 1642.782666666667,
 1455.7910000000004,
 1437.6188333333337,
 1339.0550000000003,
 1168.982166666667,
 1147.7415000000003,
 1149.8710000000003,
 1141.4530000000004,
 1041.4271666666668,
 1043.2050000000004,
 964.9795000000003,
 525.599

In [4]:
def pvpredict(predict_date) :
    path = "tongsdata/"
    filename =     predict_date.replace("_",'-') + '.json'
    
    with open(path + filename, 'r', encoding='utf-8') as fin:
        data_json = json.loads(fin.read())
    
    todayData = {time: data_json[time]['currently'] for time in data_json}
    todayData_DF = pd.DataFrame(todayData).T
    todayData_DF.index.names=['time_of_day']
    todayData_DF = todayData_DF.loc['06:30':'18:00']
    todayData_DF = todayData_DF.reset_index()
    todayData_DF['month'] = todayData_DF['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
    #todayData_DF['cloudCover'] = todayData_DF['cloudCover'].fillna(value=todayData_DF['cloudCover'].mean())
    dropcolumns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure','cloudCover']
    for i in dropcolumns:
        if i in todayData_DF.columns:
            todayData_DF = todayData_DF.drop(columns=[i])
    todayData_DF = pd.get_dummies(todayData_DF, columns=['time_of_day', 'precipType', 'uvIndex', 'month'])
    
    masterData_for_predict = masterData.drop(columns=['PV Power Generation'])
    for i in range(len(masterData_for_predict.columns)) : #drop power balance
        if masterData_for_predict.columns[i] not in todayData_DF.columns:
            todayData_DF.insert(i, masterData.columns[i+1], 0)
    missingvalue = todayData_DF[todayData_DF.isna().any(axis=1)].index
    import math
    for i in missingvalue:
        for j in todayData_DF.columns:
            if   i == 0:    #add for index error case                                         
                    todayData_DF.loc[i,j] = (todayData_DF.loc[i+1,j])
            elif i+1 > len(todayData_DF.index)-1:               
                    todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j])
            elif math.isnan(todayData_DF.loc[i,j]) :
                todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j]+todayData_DF.loc[i+1,j])/2
    first_predict = todayData_DF.astype('float').values
    
    results_first_predict = exported_pipeline.predict(first_predict)
    results_first_predict = np.array(results_first_predict).tolist()
    for i in range(25):
        results_first_predict.insert(0,0.0)
    for j in range (24):
        results_first_predict.append(0.0)
    return results_first_predict

In [22]:
predict_date = "2019_04_13"

In [24]:
path = "Testdate/" #from load into file above
filename = predict_date.replace("_",'-') + '.json'

with open(path + filename, 'r', encoding='utf-8') as fin:
    data_json = json.loads(fin.read())


todayData_DF = pd.DataFrame.from_dict(data_json)
todayData_DF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96 entries, 00:00 to 23:45
Data columns (total 18 columns):
apparentTemperature    96 non-null float64
cloudCover             96 non-null float64
dewPoint               96 non-null float64
humidity               96 non-null float64
icon                   96 non-null object
ozone                  96 non-null float64
precipIntensity        96 non-null float64
precipProbability      96 non-null float64
precipType             0 non-null object
pressure               96 non-null float64
summary                96 non-null object
temperature            96 non-null float64
time                   96 non-null int64
uvIndex                96 non-null int64
visibility             96 non-null float64
windBearing            96 non-null int64
windGust               96 non-null float64
windSpeed              96 non-null float64
dtypes: float64(12), int64(3), object(3)
memory usage: 14.2+ KB


In [26]:
todayData_DF.index.names=['time_of_day']
todayData_DF = todayData_DF.loc['06:30':'18:00']
todayData_DF = todayData_DF.reset_index()
todayData_DF['month'] = todayData_DF['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
todayData_DF

,time_of_day,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,month
0,06:30,100.19,0.14,69.58,0.45,clear-day,261.11,0.0,0.0,None,1008.28,Clear,94.14,1555135200,12,10.0,187,13.11,12.98,4
1,06:45,100.19,0.14,69.58,0.45,clear-day,261.11,0.0,0.0,None,1008.28,Clear,94.14,1555135200,12,10.0,187,13.11,12.98,4
2,07:00,99.57,0.15,68.23,0.43,clear-day,260.83,0.0,0.0,None,1007.32,Clear,94.58,1555138800,10,10.0,186,13.04,12.88,4
3,07:15,99.57,0.15,68.23,0.43,clear-day,260.83,0.0,0.0,None,1007.32,Clear,94.58,1555138800,10,10.0,186,13.04,12.88,4
4,07:30,99.57,0.15,68.23,0.43,clear-day,260.83,0.0,0.0,None,1007.32,Clear,94.58,1555138800,10,10.0,186,13.04,12.88,4
5,07:45,99.57,0.15,68.23,0.43,clear-day,260.83,0.0,0.0,None,1007.32,Clear,94.58,1555138800,10,10.0,186,13.04,12.88,4
6,08:00,98.95,0.19,67.00,0.40,clear-day,260.48,0.0,0.0,None,1006.36,Clear,94.87,1555142400,7,10.0,185,12.83,12.66,4
7,08:15,98.95,0.19,67.00,0.40,clear-day,260.48,0.0,0.0,None,1006.36,Clear,94.87,1555142400,7,10.0,185,12.83,12.66,4
8,08:30,98.95,0.19,67.00,0.40,clear-day,260.48,0.0,0.0,None,1006.36,Clear,94.87,1555142400,7,10.0,185,12.83,12.66,4
9,08:45,98.95,0.19,67.00,0.40,clear-day,260.48,0.0,0.0,None,1006.36,Clear,94.87,1555142400,7,10.0,185,12.83,12.66,4


In [27]:
#todayData_DF['cloudCover'] = todayData_DF['cloudCover'].fillna(value=todayData_DF['cloudCover'].mean())
dropcolumns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure','cloudCover']
for i in dropcolumns:
    if i in todayData_DF.columns:
        todayData_DF = todayData_DF.drop(columns=[i])
todayData_DF = pd.get_dummies(todayData_DF, columns=['time_of_day', 'precipType', 'uvIndex', 'month'])

masterData_for_predict = masterData.drop(columns=['PV Power Generation'])
for i in range(len(masterData_for_predict.columns)) : #drop power balance
    if masterData_for_predict.columns[i] not in todayData_DF.columns:
        todayData_DF.insert(i, masterData.columns[i+1], 0)
missingvalue = todayData_DF[todayData_DF.isna().any(axis=1)].index
import math
for i in missingvalue:
    for j in todayData_DF.columns:
        if   i == 0:    #add for index error case                                         
                    todayData_DF.loc[i,j] = (todayData_DF.loc[i+1,j])
        elif i+1 > len(todayData_DF.index)-1:               
                    todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j])
        elif math.isnan(todayData_DF.loc[i,j]) :
                todayData_DF.loc[i,j] = (todayData_DF.loc[i-1,j]+todayData_DF.loc[i+1,j])/2
first_predict = todayData_DF.astype('float').values

array([[100.19,   0.  ,  69.58, ...,   0.  ,   1.  ,   1.  ],
       [100.19,   0.  ,  69.58, ...,   0.  ,   1.  ,   1.  ],
       [ 99.57,   0.  ,  68.23, ...,   1.  ,   0.  ,   1.  ],
       ...,
       [ 94.54,   0.  ,  75.08, ...,   0.  ,   0.  ,   1.  ],
       [ 94.54,   0.  ,  75.08, ...,   0.  ,   0.  ,   1.  ],
       [ 94.01,   0.  ,  75.4 , ...,   0.  ,   0.  ,   1.  ]])

In [30]:
todayData_DF

,apparentTemperature,cloudCover,dewPoint,humidity,ozone,precipIntensity,precipProbability,pressure,temperature,visibility,...,uvIndex_7,uvIndex_8,uvIndex_9,month_1,month_10,month_11,month_12,uvIndex_10,uvIndex_12,month_4
0,100.19,0,69.58,0.45,0,0,0,0,94.14,10.0,...,0,0,0,0,0,0,0,0,1,1
1,100.19,0,69.58,0.45,0,0,0,0,94.14,10.0,...,0,0,0,0,0,0,0,0,1,1
2,99.57,0,68.23,0.43,0,0,0,0,94.58,10.0,...,0,0,0,0,0,0,0,1,0,1
3,99.57,0,68.23,0.43,0,0,0,0,94.58,10.0,...,0,0,0,0,0,0,0,1,0,1
4,99.57,0,68.23,0.43,0,0,0,0,94.58,10.0,...,0,0,0,0,0,0,0,1,0,1
5,99.57,0,68.23,0.43,0,0,0,0,94.58,10.0,...,0,0,0,0,0,0,0,1,0,1
6,98.95,0,67.00,0.40,0,0,0,0,94.87,10.0,...,1,0,0,0,0,0,0,0,0,1
7,98.95,0,67.00,0.40,0,0,0,0,94.87,10.0,...,1,0,0,0,0,0,0,0,0,1
8,98.95,0,67.00,0.40,0,0,0,0,94.87,10.0,...,1,0,0,0,0,0,0,0,0,1
9,98.95,0,67.00,0.40,0,0,0,0,94.87,10.0,...,1,0,0,0,0,0,0,0,0,1


In [18]:
results_first_predict = exported_pipeline.predict(first_predict)
results_first_predict = np.array(results_first_predict).tolist()
for i in range(25):
    results_first_predict.insert(0,0.0)
for j in range (24):
    results_first_predict.append(0.0)
results_first_predict

In [19]:
predictgen("2019_04_13")

ValueError: Number of features of the model must match the input. Model n_features is 73 and input n_features is 76 